In [37]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from registration_utilities import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of utilities2015 failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: __abstractmethods__
]


In [38]:
atlas_spec = dict(name='atlasV6',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

In [39]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.


In [40]:
atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol = get_structure_centroids(vol_origin_dict=atlas_structures_wrt_canonicalAtlasSpace_atlasResol)

In [41]:
atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['12N'][:2], 0]
atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['3N_L'][:2], 0]

In [72]:
# stack = 'CHATM2'
stack = 'MD590'

In [73]:
intensity_volume_spec = dict(name=stack, resolution='10.0um', prep_id='wholebrainWithMargin', vol_type='intensity')
_, (wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x, \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y, \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z) = \
DataManager.load_original_volume_v2(intensity_volume_spec, return_origin_instead_of_bbox=True)

In [74]:
if stack == 'CHATM2':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([565,246,179]) * 20. / 10. # 12N
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([388,183,179]) * 20. / 10. # 3N_L
elif stack == 'CHATM3':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([740,258,230]) * 20. / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([550,195,230]) * 20. / 10.
elif stack == 'MD593':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([647*20., 233*20., 164*10.+138*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([474*20., 171*20., 164*10.+138*20.]) / 10.
elif stack == 'MD589':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([605*20., 260*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([456*20., 175*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
elif stack == 'MD585':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([614*20., 241*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+117*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([423*20., 165*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+145*20.]) / 10.
elif stack == 'MD590':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([650*20., 239*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([473*20., 159*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
else:
    raise

In [75]:
def align_anchors(pm1, pm2, pf1, pf2):
    """
    Align the vector (pm1, pm2) and vector (pf1, pf2), and align the two middle points. 
    """
    
    t = ((pf1 + pf2) / 2. - (pm1 + pm2) / 2.)

    subject_d = pf1 - pf2
    atlas_d = pm1 - pm2

    subject_d_n = subject_d / np.linalg.norm(subject_d)
    atlas_d_n = atlas_d / np.linalg.norm(atlas_d)

    R = R_align_two_vectors(atlas_d_n, subject_d_n)

    T = np.zeros((3,4))
    T[:3, :3] = R
    T[:3, 3] = t
    
    return T

In [76]:
wholebrainWithMargin_origin_wrt_wholebrain_dataResol = \
np.r_[wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x,
      wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y,
      0]

In [77]:
subject_anchor1_wrt_wholebrain_atlasResol = subject_anchor1_wrt_wholebrainWithMargin_atlasResol + \
wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

subject_anchor2_wrt_wholebrain_atlasResol = subject_anchor2_wrt_wholebrainWithMargin_atlasResol + \
wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

In [78]:
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = align_anchors(atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol, atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol,
             subject_anchor1_wrt_wholebrain_atlasResol, subject_anchor2_wrt_wholebrain_atlasResol)

In [79]:
print T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol

[[ 9.98572232e-01  5.34281308e-02  0.00000000e+00  1.26609838e+03]
 [-5.34281308e-02  9.98572232e-01  0.00000000e+00  5.59968802e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  4.10000000e+02]]


In [80]:
bp.pack_ndarray_file(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, 
                     '/home/yuncong/' + stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.bp')